In [1]:
import os, sys
import numpy as np

from pathlib import Path

from astropy.io import fits, ascii
from astropy.table import Table, Column, vstack, hstack, join

from scipy import interpolate
import scipy.stats as stats
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline

sys.path.append("./py")
from utils import *

sys.path.append("/Users/aberti/Desktop/research")
from plotutils import get_corners, fig_labels, get_colors, plot_settings
plt.rcParams.update(**plot_settings)

plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.family"] = "STIXGeneral"

from params import BASEDIR, DATADIR, SIMDIR, H0, Om0
from params import get_boxsize, get_zsnap_data, get_sham_var_bins, get_abs_mag_lim, get_abs_mag_bins_clust

#-- bins for clustering (data and mocks)
from params import H0, bin_file_comoving, bin_file

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

from functions import get_survey_area

import Corrfunc
from Corrfunc.theory.wp import wp as wp_corrfunc


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:550: UserWarning: matplotlibrc text.usetex can not be used unless ghostscript-9.00 or later is installed on your system
  % gs_req)


# Set parameters


In [2]:
band   = "Mz"
zrange = (0.4,0.5)

rp_use_range = (0.1, None)

zmag_lim = 20.7
sim_tag  = "mdpl2"
sham_tag = "vpeak"
d        = "south"
pimax    = 150
h        = H0/100.
print(f"h = {h}")

boxsize = get_boxsize(sim_tag)

#========== BEGIN HOUSEKEEPING ==========#
z_snaps, snap_nums, snaps = get_zsnap_data(sim_tag)

hist_idx = [len(snaps)-i for i in snap_nums]
vol_sim  = get_boxsize(sim_tag)**3
a_snaps  = [snaps["scale"][snaps["snapnum"]==sn].data[0] for sn in snap_nums]

sham_var_bins, sham_var_cens = get_sham_var_bins(sham_tag)

zmin, zmax  = zrange
cat_tag     = get_cat_tag(d,zrange)
abs_mag_lim = get_abs_mag_lim(zmin, band)
zsim        = np.array(z_snaps)[ np.round(z_snaps,1)==zmin ][0]
zsnap_tag   = get_zsnap_tag(zsim)
zmag_tag    = get_zmag_tag(zmag_lim)

MW1lim = get_abs_mag_lim(zmin,"MW1")
Mrlim  = get_abs_mag_lim(zmin,"Mr")
Mzlim  = get_abs_mag_lim(zmin,"Mz")
if band=="MW1":
    abs_mag_lim_tag = get_MW1_tag(MW1lim)
elif band=="Mr":
    abs_mag_lim_tag = get_Mr_tag(Mrlim)
elif band=="Mz":
    abs_mag_lim_tag = get_Mz_tag(Mzlim)
else:
    raise Exception()

n_mag_bins_clust   = 4
abs_mag_bins_clust = get_abs_mag_bins_clust(zmin, band, nbins=n_mag_bins_clust)
abs_mag_bin_tags   = get_abs_mag_bin_tags(zmin, band, nbins=n_mag_bins_clust)

lf_name  = f"{BASEDIR}/data/stats/lum_func_{band}_{cat_tag}_{zmag_tag}_{abs_mag_lim_tag}.npy"
lum_func_full = np.load(f"{BASEDIR}/data/stats/lum_func_{band}_{cat_tag}_{zmag_tag}_full.npy").item()
#=========== END HOUSEKEEPING ===========#

# try:
#     t
# except:
#     t = Table(np.load(f"{BASEDIR}/mags_z0p40-0p70.npy"))

# pz_mask = ((t["pz"]) >= zmin) & (t["pz"] < zmax)

# abs_mag_bin_cens   = [np.mean(abs_mag_bins_clust[i:i+2]) for i in range(len(abs_mag_bins_clust)-1)]
# abs_mag_bin_means  = [np.round(  np.mean( t[band][pz_mask & (t[band] < abs_mag_bins_clust[i+1]) & (t[band] >= abs_mag_bins_clust[i])] ),3) for i in range(len(abs_mag_bins_clust)-1)]
# abs_mag_bin_meds   = [np.round(np.median( t[band][pz_mask & (t[band] < abs_mag_bins_clust[i+1]) & (t[band] >= abs_mag_bins_clust[i])] ),3) for i in range(len(abs_mag_bins_clust)-1)]
# abs_mag_bin_counts = np.array([      len( t[band][pz_mask & (t[band] < abs_mag_bins_clust[i+1]) & (t[band] >= abs_mag_bins_clust[i])] )    for i in range(len(abs_mag_bins_clust)-1)])

print(band)
print(zrange)

# abs_mag_bin_count_weights = abs_mag_bin_counts/np.sum(abs_mag_bin_counts)

# print(abs_mag_bin_cens)
# print(abs_mag_bin_means)
# print(abs_mag_bin_meds)
# print(abs_mag_bin_counts)
# print(abs_mag_bin_count_weights)

params = dict(band=band,
              zrange=zrange, zmin=zmin, zmax=zmax,
              rp_use_range=rp_use_range,
              #brightest_mag_bin_rp1Mpch=brightest_mag_bin_rp1Mpch,
              zmag_lim=zmag_lim,
              sim_tag=sim_tag,
              sham_tag=sham_tag,
              d=d,
              pimax=pimax,
              h=h,
              z_snaps=z_snaps,
              snap_nums=snap_nums,
              a_snaps=a_snaps,
              snaps=snaps,
              hist_idx=hist_idx,
              vol_sim=vol_sim,
              sham_var_bins=sham_var_bins,
              sham_var_cens=np.round(sham_var_cens,3),
              cat_tag=cat_tag,
              abs_mag_lim=abs_mag_lim,
              zsim=zsim,
              zsnap_tag=zsnap_tag,
              zmag_tag=zmag_tag,
              Mzlim=Mzlim,
              MW1lim=MW1lim,
              abs_mag_lim_tag=abs_mag_lim_tag,
              n_mag_bins_clust=n_mag_bins_clust,
              abs_mag_bins_clust=abs_mag_bins_clust,
              abs_mag_bin_tags=abs_mag_bin_tags,
#              lum_func=lum_func,
              lum_func_full=lum_func_full,
#               pz_mask=pz_mask,
#               abs_mag_bin_cens=abs_mag_bin_cens,
#               abs_mag_bin_means=abs_mag_bin_means,
#               abs_mag_bin_counts=abs_mag_bin_counts,
#               abs_mag_bin_count_weights=abs_mag_bin_count_weights,
             )

def param_list(values=False):
    for k in params.keys():
        ktype = str(type(params[k]))[8:-2]
        if values:
            if ktype in ("str","float","int","numpy.float64","numpy.ndarray","list","tuple","bool"):
                print(f"{k} ({ktype})\n{params[k]}\n")
            elif ktype=="dict":
                print(f"{k} ({ktype})\n{list(params[k].keys())}\n")
            elif "table" in ktype:
                print(f"{k} (table)\n{params[k].colnames}\n")
            else:
                print(f"{k} ({ktype})\n")
        else:
            print(f"{k}")



h = 0.6777
Mz
(0.4, 0.5)


<a class="anchor" id="load_mock"></a>

# Load mock

[BACK TO TOP](#outline)


In [3]:
%%time

mock_fname = f"{BASEDIR}/mocks/{sim_tag}/{sham_tag}/{d}/{zsnap_tag}_{abs_mag_lim_tag}.npy"

print(f"Loading {mock_fname}...")

halocat = Table(np.load(mock_fname))


Loading /Users/aberti/Desktop/research/desi_lrg_sham/mocks/mdpl2/vpeak/south/zsnap0p42531_Mzlimn21p6.npy...
CPU times: user 6.49 s, sys: 13.4 s, total: 19.9 s
Wall time: 28.6 s


In [4]:
abs_mag_bins = np.round(lum_func_full["abs_mag_bins"],2)

# print( np.array(lum_func_full["ng_eff"]) )
# print( np.array(lum_func_full["ng_eff"])[np.where(abs_mag_bins <= abs_mag_lim)] )

print("max galaxy number density")
print( np.max( np.array(lum_func_full["ng_eff"]) ) )
print("galaxy number density at magnitude limit [Mpc^(-3)]:")
print( np.max( np.array(lum_func_full["ng_eff"])[np.where(abs_mag_bins <= abs_mag_lim)] ) )

# print(int(np.max( np.array(lum_func_full["ng_eff"]) ) * (vol_sim/(h**3)) ))
Nhalo_select = int(np.max( np.array(lum_func_full["ng_eff"])[np.where(abs_mag_bins <= abs_mag_lim)] ) * (vol_sim/(h**3)) )

print("\nnumber of halos to select to achieve target number density:")
print(Nhalo_select)

print(len(halocat[halocat[f"{band}_no_scatter"] <= abs_mag_lim]))


max galaxy number density
0.003979603222261111
galaxy number density at magnitude limit [Mpc^(-3)]:
0.0018898983721830406

number of halos to select to achieve target number density:
6071919
6486602


In [ ]:
# %%time

# sigma_abs_mag = 0.6

# sc = str(round(sigma_abs_mag,2)).replace(".","p")
# sigma_sham_tag = f"sigma_sham_const{sc}"

# abs_mag_cname = f"{band}_sig{str(np.round(sigma_abs_mag,2)).replace('.','p')}"
# print(abs_mag_cname)

# this_galcat = halocat[ halocat[abs_mag_cname] <= abs_mag_lim ]

# #-- sort catalog by model magnitude
# this_galcat.sort( abs_mag_cname )
# assert(this_galcat[abs_mag_cname][0] < this_galcat[abs_mag_cname][-1])

# #-- add column for galaxy flag
# if "galaxy" not in this_galcat.colnames:
#     this_galcat.add_column( Column(np.zeros(len(this_galcat),dtype=bool), name="galaxy"))

# #-- flag halos with galaxies
# this_galcat["galaxy"][:Nhalo_select] = True

# this_galcat = this_galcat[ this_galcat["galaxy"]==True ]

# plt.hist(this_galcat[f"{band}_no_scatter"], bins=abs_mag_bins)
# plt.hist(this_galcat[abs_mag_cname], bins=abs_mag_bins, histtype="step", lw=3)

# plt.show()


In [5]:
def _mock_wp_fast(cat, boxsize=None, sigma_los=None, nthreads=1, pimax=150., bin_file=None, N_iter=1, down_sample=False, quiet=False):
    """
    Corrfunc wp (autocorrelation) calculation of unpadded box using analytic randoms
    """
    out = []
    for (u,v,w) in (("x","y","z"),("y","z","x"),("z","x","y")):
        xx, yy, zz = cat[u], cat[v], cat[w]

        z_scatter = np.array([np.random.normal(zz, sigma_los, len(zz)) for idx in range(N_iter)])
        z_scatter[z_scatter < 0] = np.array(boxsize - np.abs(z_scatter[z_scatter < 0])%boxsize)
        z_scatter[z_scatter > boxsize] = np.array(z_scatter[z_scatter > boxsize]%boxsize)

        if type(down_sample)==int:
            ds = down_sample
            out.append([wp_corrfunc(boxsize, pimax, nthreads, bin_file, xx[::ds], yy[::ds], z_s[::ds], output_rpavg=False)["wp"] for z_s in z_scatter])
        else:
            out.append([wp_corrfunc(boxsize, pimax, nthreads, bin_file, xx, yy, z_s, output_rpavg=False)["wp"] for z_s in z_scatter])

    out = np.concatenate(out).T

    return out


def _run_mock_wp_fast(cat=None, BASEDIR=None, sim_tag=None, zmag_tag=None, abs_mag_lim_tag=None,
                      abs_mag_bin_tag=None, sigma_sham_tag=None, chunk=None, zsnap_tag=None, d=None, pimax=None,
                      sigma_los_const_array=None, quiet=True):
    save_path = f"{BASEDIR}/clustering/mocks/{sim_tag}/{sham_tag}/{zmag_tag}/{abs_mag_lim_tag}/{abs_mag_bin_tag}/{sigma_sham_tag}"
    Path( save_path ).mkdir(parents=True, exist_ok=True)

    save_as = f"{save_path}/{zsnap_tag}_{d}_GXG_pimax{int(pimax)}.npy"
    if os.path.exists( save_as ):
        wp_all_sigma_los = np.load(save_as, allow_pickle=True).item()
    else:
        wp_all_sigma_los = {}
    for sigma_los in sigma_los_const_array:
        key = f"sigma_los_{sigma_los}"
        if key in wp_all_sigma_los.keys():
            if not quiet:
                print(f"sigma_los = {sigma_los} Mpc/h found; skipping...")
        else:
            wp_this_sigma_los = _mock_wp_fast(cat, boxsize=boxsize, pimax=pimax, bin_file=bin_file_comoving,
                                              quiet=quiet, sigma_los=sigma_los)
            wp  = np.mean(wp_this_sigma_los, axis=1)
            std = np.std(wp_this_sigma_los, axis=1)
            wp_all_sigma_los[key] = dict(rp_mids=rp_mids, wp=wp, std=std)
    np.save( save_as, wp_all_sigma_los, allow_pickle=True )
    

In [6]:
%%time

cross = "GXG"
quiet = False

clust_mag_binned = True
clust_full_mock  = False

rp_bins, rp_mids = rp_bins_from_file(bin_file_comoving)

# sigma_los_const_array = np.arange(50,151,10) # COARSE
sigma_los_const_array = np.arange(50,121,5) # FINE

# for sigma_abs_mag in np.arange(0.1,1.51,0.1): # COARSE
for sigma_abs_mag in np.arange(0.5,0.701,0.01): # FINE
    sc = str(round(sigma_abs_mag,2)).replace(".","p")
    sigma_sham_tag = f"sigma_sham_const{sc}"
    
    ##########################################
    ##  Flag (sub)halos with mock galaxies  ##
    ##########################################
    
    abs_mag_cname = f"{band}_sig{str(np.round(sigma_abs_mag,2)).replace('.','p')}"
    print(f"{now()} {abs_mag_cname}")

    this_galcat = halocat[ halocat[abs_mag_cname] <= abs_mag_lim ]
    # print(len(this_galcat))
    
    #-- sort catalog by model magnitude
    this_galcat.sort( abs_mag_cname )
    assert(this_galcat[abs_mag_cname][0] < this_galcat[abs_mag_cname][-1])

    #-- add column for galaxy flag
    if "galaxy" not in this_galcat.colnames:
        this_galcat.add_column(Column(np.zeros(len(this_galcat),dtype=bool), name="galaxy"))

    #-- flag halos with galaxies
    this_galcat["galaxy"][:Nhalo_select] = True
    
    this_galcat = this_galcat[ this_galcat["galaxy"]==True ]
    # print(len(this_galcat))

    kwargs = dict(BASEDIR=BASEDIR, sim_tag=sim_tag, zmag_tag=zmag_tag, abs_mag_lim_tag=abs_mag_lim_tag,
                  sigma_sham_tag=sigma_sham_tag, zsnap_tag=zsnap_tag, d=d, pimax=pimax,
                  sigma_los_const_array=sigma_los_const_array, quiet=quiet)

    if clust_mag_binned:
        abs_mag = this_galcat[abs_mag_cname]
        for i in range(len(abs_mag_bins_clust)-1):
            mag_min, mag_max = abs_mag_bins_clust[i:i+2]
            abs_mag_bin_tag  = abs_mag_bin_tags[i]
            if not quiet:
                print(f"\n{now()} ...{band} bin {i+1} / {len(abs_mag_bins_clust)-1}: {abs_mag_bin_tag}")
            mag_mask = (abs_mag > mag_min) & (abs_mag <= mag_max)
            if not quiet:
                print(f"N = {len( this_galcat[mag_mask] )}")
            kwargs["cat"] = this_galcat[mag_mask]
            kwargs["abs_mag_bin_tag"] = abs_mag_bin_tag
            _run_mock_wp_fast(**kwargs)

    if clust_full_mock:
        abs_mag_bin_tag = "no_abs_mag_bins"
        if not quiet:
            print(f"{now()} ...full mock: {abs_mag_bin_tag}")
        kwargs["cat"] = this_galcat      
        kwargs["abs_mag_bin_tag"] = abs_mag_bin_tag        
        _run_mock_wp_fast(**kwargs)



[08:36:27] Mz_sig0p5

[08:36:47] ...Mz bin 1 / 4: Mzn30p0-n22p35
N = 1751113
sigma_los = 50 Mpc/h found; skipping...
sigma_los = 60 Mpc/h found; skipping...
sigma_los = 70 Mpc/h found; skipping...
sigma_los = 80 Mpc/h found; skipping...
sigma_los = 90 Mpc/h found; skipping...
sigma_los = 100 Mpc/h found; skipping...
sigma_los = 110 Mpc/h found; skipping...
sigma_los = 120 Mpc/h found; skipping...

[08:40:24] ...Mz bin 2 / 4: Mzn22p35-n22p1
N = 1135505
sigma_los = 50 Mpc/h found; skipping...
sigma_los = 60 Mpc/h found; skipping...
sigma_los = 70 Mpc/h found; skipping...
sigma_los = 80 Mpc/h found; skipping...
sigma_los = 90 Mpc/h found; skipping...
sigma_los = 100 Mpc/h found; skipping...
sigma_los = 110 Mpc/h found; skipping...
sigma_los = 120 Mpc/h found; skipping...

[08:42:14] ...Mz bin 3 / 4: Mzn22p1-n21p85
N = 1586898
sigma_los = 50 Mpc/h found; skipping...
sigma_los = 60 Mpc/h found; skipping...
sigma_los = 70 Mpc/h found; skipping...
sigma_los = 80 Mpc/h found; skipping...
sigma

sigma_los = 110 Mpc/h found; skipping...
sigma_los = 120 Mpc/h found; skipping...

[16:41:39] ...Mz bin 3 / 4: Mzn22p1-n21p85
N = 1586898
sigma_los = 50 Mpc/h found; skipping...
sigma_los = 60 Mpc/h found; skipping...
sigma_los = 70 Mpc/h found; skipping...
sigma_los = 80 Mpc/h found; skipping...
sigma_los = 90 Mpc/h found; skipping...
sigma_los = 100 Mpc/h found; skipping...
sigma_los = 110 Mpc/h found; skipping...
sigma_los = 120 Mpc/h found; skipping...

[16:44:32] ...Mz bin 4 / 4: Mzn21p85-n21p6
N = 1598403
sigma_los = 50 Mpc/h found; skipping...
sigma_los = 60 Mpc/h found; skipping...
sigma_los = 70 Mpc/h found; skipping...
sigma_los = 80 Mpc/h found; skipping...
sigma_los = 90 Mpc/h found; skipping...
sigma_los = 100 Mpc/h found; skipping...
sigma_los = 110 Mpc/h found; skipping...
sigma_los = 120 Mpc/h found; skipping...
CPU times: user 7h 55min 56s, sys: 8min 12s, total: 8h 4min 9s
Wall time: 8h 10min 58s
